# Instructions:
1) In 'Specify Input/ Output Directories' code block, update: INPUT_PATH_FOLDER and OUTPUT_PATH_FOLDER
2) Run all cells in notebook

If you are having issue (or are working on this code) see the developer notes section at the end

## Import Libraries/ Functions:

In [13]:
#general library:
import os
import shutil

#add sextractor to path
import sys
sys.path.insert(0, '../gibbi')

#now we can import the preprocess function:
from fits import preprocess_fits, write_fits, read_fits

## Specify Input/ Output Directories:

In [14]:
################################  IMPORTANT ################################:
#modify these paths:

#path to folder (as it appears on your native OS)
FITS_PATH_FOLDER =  '../../spin-parity-catalog/galaxies/table5'
PREPROCESS_OUTPUT_PATH_FOLDER = '../../spin-parity-catalog/image_processing/preprocess_fits/table5'
BAD_PIXEL_OUTPUT_PATH_FOLDER = '../../spin-parity-catalog/image_processing/bad_pixel_mask/table5'

#########################################################################

In [15]:
#checks fits path:
if not os.path.isdir(FITS_PATH_FOLDER):
    print('FITS_PATH_FOLDER = {}: does not exist, double check the FITS_PATH_FOLDER variable (HINT: you have to modify this)'.format(FITS_PATH_FOLDER))
    
#check if preprocess output directory exists, and if not make it:
if os.path.isdir(PREPROCESS_OUTPUT_PATH_FOLDER):
    pass
elif PREPROCESS_OUTPUT_PATH_FOLDER != '':
    os.makedirs(PREPROCESS_OUTPUT_PATH_FOLDER)
else:
    print('PREPROCESS_OUTPUT_PATH_FOLDER = {}: is blank, double check the PREPROCESS_OUTPUT_PATH_FOLDER variable (HINT: you have to modify this)'.format(PREPROCESS_OUTPUT_PATH_FOLDER))
  
#check if bad pixel output directory exists, and if not make it:
if os.path.isdir(BAD_PIXEL_OUTPUT_PATH_FOLDER):
    pass
elif BAD_PIXEL_OUTPUT_PATH_FOLDER != '':
    os.makedirs(BAD_PIXEL_OUTPUT_PATH_FOLDER)
else:
    print('BAD_PIXEL_OUTPUT_PATH_FOLDER = {}: is blank, double check the BAD_PIXEL_OUTPUT_PATH_FOLDER variable (HINT: you have to modify this)'.format(BAD_PIXEL_OUTPUT_PATH_FOLDER))
   
 

## Preprocess and create Bad Pixel Mask files for all fits in nested INPUT_PATH_FOLDER:

In [16]:
def get_all_fits_in_folder(folder_path):
    fits_files = []
    for file in os.listdir(folder_path):
        if file.endswith(".fits") and "mask" not in file:
            fits_files.append(file)
    return fits_files

In [17]:
def run_preprocess_and_bad_with_all_fits(fits_folder,preprocess_folder,bad_pixel_folder):
    for gal_name in os.listdir(fits_folder):
        
        #Step 1: Create Output dir for specific galaxy
        print('Running on {}'.format(gal_name))
        gal_output_path = os.path.join(preprocess_folder,gal_name)
        os.makedirs(gal_output_path,exist_ok=True) #requires python 3.2+
        
        gal_output_path = os.path.join(bad_pixel_folder,gal_name)
        os.makedirs(gal_output_path,exist_ok=True) #requires python 3.2+
        
        #Step 2: Get all sextractor files for galaxy
        fits_files =  get_all_fits_in_folder(os.path.join(fits_folder,gal_name))
        
        #Step 3: Iterate through fits and run_sextractor on all fits
        for the_fits in fits_files:
            #Step 3a: Get input_fits_path, output_preprocess_path, and output_bad_pixel_path
            # i.e:
            #     for fits file 'IC1683/IC1683_g.fits' in directory <fits_folder>:
            #     input_fits_path = '<fits_folder>/IC1683/IC1683_g.fits'
            #     output_preprocess_path = '<preprocess_folder>/IC1683/IC1683_g_preprocessed.fits'
            #     fits_path = '<bad_pixel_folder>/IC1683/IC1683_g_bad_pixel_folder.fits'
            
            preprocess_name = "{}_preprocessed.fits".format(the_fits.split('.')[0])
            bad_pixel_name = "{}_bad_pixel_mask.fits".format(the_fits.split('.')[0])
            
            input_fits_path = os.path.join(fits_folder,gal_name,the_fits)
            output_preprocess_path = os.path.join(preprocess_folder,gal_name,preprocess_name)
            output_bad_pixel_path = os.path.join(bad_pixel_folder,gal_name,bad_pixel_name)
            
            #Step 3b: Read in fits image, create preprocess fits, and bad_pixel mask
            the_fits = read_fits(input_fits_path)
            preprocessed_fits,bad_pixel_mask = preprocess_fits(the_fits,offset=10)
            
            #Step 3c: Save preprocessed fits and bad_pixel_mask
            write_fits(output_preprocess_path,preprocessed_fits)
            write_fits(output_bad_pixel_path,bad_pixel_mask)

In [18]:
run_preprocess_and_bad_with_all_fits(FITS_PATH_FOLDER,PREPROCESS_OUTPUT_PATH_FOLDER,BAD_PIXEL_OUTPUT_PATH_FOLDER)

Running on IC2247
Running on IC540
Running on IC944
Running on MCG-02-02-040
Running on MCG-02-03-015
Running on NGC1542
Running on NGC3067
Running on NGC3079
Running on NGC3169
Running on NGC3495
Running on NGC3626
Running on NGC4517
Running on NGC4605
Running on NGC4772
Running on NGC6314
Running on NGC681
Running on UGC10205
Running on UGC10297
Running on UGC3107
Running on UGC5111
Running on UGC5498
Running on UGC6036
Running on UGC8267
Running on UGC8778
Running on UGC9665


In [7]:
#Check size of input and output (and figure out if there was an issue running on something)
def compare_input_and_output(fits_folder,preprocess_folder,bad_pixel_folder):
    input_count = 0
    output_preprocess_count = 0
    output_bad_pixel_count = 0
    for gal_name in os.listdir(fits_folder):
        for file in os.listdir(os.path.join(fits_folder,gal_name)):
            if file.endswith(".fits") and "mask" not in file:
                input_count += 1
                
                preprocess_name = "{}_preprocessed.fits".format(file.split('.')[0])
                bad_pixel_name = "{}_bad_pixel_mask.fits".format(file.split('.')[0])
                
                output_preprocess = os.path.join(preprocess_folder,gal_name,preprocess_name)
                output_badpixel = os.path.join(bad_pixel_folder,gal_name,bad_pixel_name)
                
                if os.path.exists(output_preprocess):
                    output_preprocess_count += 1
                else:
                    print('Missing {}'.format(preprocess_name))
                    
                if os.path.exists(output_badpixel):
                    output_bad_pixel_count += 1
                else:
                    print('Missing {}'.format(bad_pixel_name))
    print("Input: {} fits files, Output: {} preprocess; {} bad pixel mask".format(input_count,output_preprocess_count,output_bad_pixel_count))

compare_input_and_output(FITS_PATH_FOLDER,PREPROCESS_OUTPUT_PATH_FOLDER,BAD_PIXEL_OUTPUT_PATH_FOLDER)

Input: 675 fits files, Output: 675 preprocess; 675 bad pixel mask


## Developer Notes:
Last Updated: 06/29/2022
 
TODO: Write